In [1]:
import calcbench as cb

In [8]:
cb.dimensional_raw(company_identifiers=['fdx'], metrics=['OperatingSegmentRevenue'], start_year=2018)

[{'value': 37331000000.0,
  'is_subtotal': False,
  'dimensions': {'Segment': 'FedEx Express Segment - Operating Segments'},
  'container': 'FedexExpressSegmentMember,OperatingSegmentsMember',
  'metric': 'OperatingSegmentRevenue',
  'calendar_year': 2018,
  'calendar_period': 0,
  'fiscal_year': 2019,
  'fiscal_period': 0,
  'trace_facts': [{'local_name': 'RevenuefromContractwithCustomerExcludingAssessedTax',
    'negative_weight': False,
    'XBRL_fact_value': None,
    'fact_id': 195903006,
    'dimensions': None}],
  'ticker': 'FDX',
  'calcbench_entity_id': 545},
 {'value': 20522000000.0,
  'is_subtotal': False,
  'dimensions': {'Segment': 'FedEx Ground Segment - Operating Segments'},
  'container': 'FedexGroundSegmentMember,OperatingSegmentsMember',
  'metric': 'OperatingSegmentRevenue',
  'calendar_year': 2018,
  'calendar_period': 0,
  'fiscal_year': 2019,
  'fiscal_period': 0,
  'trace_facts': [{'local_name': 'RevenuefromContractwithCustomerExcludingAssessedTax',
    'negative

In [5]:
cb.dimensional_raw?